In [1]:
# import Dependancies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Create a set of random latitube and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats,lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
coordinates

[(25.549429994495867, -89.56786835506541),
 (39.76278018731864, -160.41077168804722),
 (50.68077341327901, 172.4075217736654),
 (-20.22319628190033, -113.39237964179497),
 (88.22199917598539, -19.125597794641976),
 (44.73165434266639, 162.9343741445993),
 (26.12474329639643, -13.80362619614823),
 (60.637475088342114, 170.66757565937115),
 (52.05643212406045, -68.17088251977157),
 (-35.22062043259212, -105.38744594041245),
 (-13.45854582619586, 157.78539697015208),
 (25.141061842455358, 44.20572632550528),
 (-51.091164852339034, 149.25430113871153),
 (-47.241692100400165, 125.56779831931362),
 (72.33630256161328, 160.46333854502666),
 (50.44910908747477, -118.41197190589247),
 (80.06915680161364, 42.52535120215276),
 (-9.462452070725632, 31.296078380311258),
 (-32.81414376911126, -43.46578476628824),
 (10.664286875901112, 39.773573809062725),
 (12.99147086139051, -165.72876205629223),
 (35.77443185510913, -39.28533204429132),
 (63.670672751546476, -13.213011051323122),
 (24.763346978622

In [5]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

610